In [1]:
# 使用tensorflow的keras接口来建模
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint
import jieba
import re
from gensim.models import KeyedVectors
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Dense, LSTM, Embedding
import numpy as np

C:\SoftWare\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
# 使用gensim加载预训练中文分词embedding, 有可能需要等待1-2分钟
cn_model = KeyedVectors.load_word2vec_format('../embeddings/sgns.zhihu.bigram', binary=False, unicode_errors="ignore")

W0709 18:51:24.109871 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:26.823616 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:32.202240 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:36.158664 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:37.428308 15128 utils_any2vec.py:185] duplicate word '----------------------------------

W0709 18:51:41.552248 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:41.560227 15128 utils_any2vec.py:185] duplicate word '..................................................................................................' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:41.564217 15128 utils_any2vec.py:185] duplicate word '--------------------------------------------------------------------------------------------------' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:41.572194 15128 utils_any2vec.py:185] duplicate word '..................................................................................................' in ../embeddings/sgns.zhihu.bigram, ignoring all but first
W0709 18:51:41.574189 15128 utils_any2vec.py:185] duplicate word '----------------------------------

In [33]:
# 用LSTM对样本进行分类
model = Sequential()

In [35]:
# 只使用前50000个词
num_words = 50000
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))

In [36]:
# 模型第一层为embedding
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=223, trainable=False))

# model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
# model.add(LSTM(units=16, return_sequences=False))
model.add(LSTM(units=32, return_sequences=False))

model.add(Dense(1, activation='sigmoid'))

In [37]:
# 建立一个权重的存储点
path_checkpoint = './sentiment_checkpoint_Class2.keras'
model.load_weights(path_checkpoint)

In [38]:
def predict_sentiment(text):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = jieba.cut(text)
    cut_list = [i for i in cut]
    # tokenize
    for i, word in enumerate(cut_list):
        try:
            cut_list[i] = cn_model.vocab[word].index
            if cut_list[i] >= 50000:
                cut_list[i] = 0
        except KeyError:
            cut_list[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_list], maxlen=223, padding='pre', truncating='pre')
    # 预测
    result = model.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('是一例正面评价', 'output=%.2f' % coef)
    else:
        print('是一例负面评价', 'output=%.2f' % coef)

In [46]:
text = "酒店环境很差"
print(predict_sentiment(text))

酒店环境很差
是一例负面评价 output=0.17
None
